In [ ]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
FOLDER = 'plots/'

def full_frame(width=None, height=None):
    ''' Nearly completely remove all borders from a plot. '''
    import matplotlib as mpl
    mpl.rcParams['savefig.pad_inches'] = 0
    figsize = None if width is None else (width, height)
    fig = plt.figure(figsize=figsize)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.autoscale(tight=True)
    return fig, ax
    
def plot_matrix(A, saveas='', cmap='RdBu', vmin=None, vmax=None):
    fig, ax = full_frame(A.shape[1], A.shape[0])
    ax.matshow(A, cmap=cmap, vmin=vmin, vmax=vmax)
    
    if saveas != '':
        plt.savefig(FOLDER + saveas)

## Setup

In [ ]:
kernel = np.array([[-1, 2], [-3, -4]])
S = np.array(image.shape)
np.random.seed(1)
image = np.random.uniform(0, 1.0, size=S)
image[0, 0] = 1.0
image[-1, -1] = 0.0

image = np.array([[1, 2, 3],[4, 5, 6],[-7, 8, 9],[-1, -2, 3]])

print(image)
print(kernel)
plot_matrix(image, 'image.eps', 'gray')

#kernel = np.array([[-1, 1]])
plot_matrix(kernel, 'k.eps', vmax=0)

## Original 2D convolution

In [ ]:
from scipy.signal import convolve2d

result_orig = convolve2d(image, kernel, mode='valid', boundary='symm')

# make sure the min and max of matrix plots are always the same. 
vmin = np.min(result_orig)
vmax = np.max(result_orig)
plot_matrix(result_orig, 'k.eps', 'gray', vmin=vmin, vmax=vmax)
print(result_orig)

## Vectorized 2D convolution

In [ ]:
from scipy.linalg import circulant

## Setup
n = S[0]*S[1]

kernel_vect = np.zeros(S)
kernel_vect[:kernel.shape[0], :kernel.shape[1]] = kernel[::-1, ::-1]
kernel_vect.resize([1, n])
plot_matrix(kernel_vect, '', vmax=2)

# Because of the convention of scipy's circulant implementation we need to undo their flipping, thus the transpose. 
kernel_matrix = np.zeros((n, n))

kernel_matrix = circulant(kernel_vect).T
plot_matrix(kernel_matrix, 'k_matrix.eps', vmax=2)

image_vect = np.reshape(image, [-1, 1])
plot_matrix(image_vect, 'image_vect.eps', 'gray')
plot_matrix(image_vect.T, 'image_vect_t.eps', 'gray')

In [ ]:
## Convolution


result_vect_vect = kernel_matrix.dot(image_vect)
plot_matrix(result_vect_vect, 'result_vect_vect.eps', 'gray', vmin=vmin, vmax=vmax)
plot_matrix(result_vect_vect.T, 'result_vect_vect_T.eps', 'gray', vmin=vmin, vmax=vmax)

result_vect = result_vect_vect.reshape(S)
plot_matrix(result_vect, 'resul_vect.eps', 'gray', vmin=vmin, vmax=vmax)

assert np.allclose(result_vect[:result_orig.shape[0], :result_orig.shape[1]], result_orig)

## 1D Fourier-domain implementation

In [ ]:
kernel_new = np.zeros(S)
kernel_new[:kernel.shape[0], :kernel.shape[1]] = kernel
kernel_new.resize([1, n])
#plot_matrix(kernel_vect, '', vmax=2)

# Because of the convention of scipy's circulant implementation we need to undo their flipping, thus the transpose. 
kernel_matrix_new = np.zeros((n, n))
kernel_matrix_new = circulant(kernel_new).T

from psf2otf import zero_pad

kernel_matrix_new = zero_pad(kernel_matrix_new, [n, n])

# TODO why do we need this factor? 
kernel_matrix_fft = np.fft.fft2(kernel_matrix_new / n)
kernel_matrix_fft[np.abs(np.real(kernel_matrix_fft)) < 1e-10] = 0.0
kernel_matrix_fft[np.abs(np.imag(kernel_matrix_fft)) < 1e-10] = 0.0

# TODO why do we need this? 
print(kernel)
kernel_matrix_fft[0, 0] = -6.0
kernel_matrix_fft[6, 6] = -4.0

real = [[-6. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    2.7321],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   5.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., -5. ,   0. ,     0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   , 0.     ,-3., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    ,  0.   ,-0.7321 , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0.,  0.    , -4.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0, 0., -0.7321,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0,  0,-3.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  0, -5, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,0.    ,  5,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ],
        [ 0. ,2.7321,  0,  0, 0.,  0.    ,  0.   , 0.     , 0., 0.  ,   0.  ,    0.    ]],

imag = [[ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    , -5.4641],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     5.1962,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 5. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.    , -1.7321 , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 1.4641 , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    0.    , -1.4641,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    ,  0.  ,    1.7321,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    ,  0.    , -5.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,0.    , -5.1962,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ],
        [ 0. ,5.4641,  0.    ,  0.  ,    0.    ,  0.    ,  0.   , 0.     , 0.     , 0. ,     0.    ,  0.    ]]

matlab_kF = np.zeros((n, n), dtype=np.complex)
matlab_kF.real = real
matlab_kF.imag = imag

plt.matshow(np.real(matlab_kF))
plt.colorbar(); plt.show()
plt.matshow(np.real(kernel_matrix_fft))
plt.colorbar(); plt.show()
plt.matshow(np.imag(matlab_kF))
plt.colorbar(); plt.show()
plt.matshow(np.imag(kernel_matrix_fft))
plt.colorbar(); plt.show()

print('not equal elements:')
diff = matlab_kF -  kernel_matrix_fft
print(diff[np.abs(diff)>1e-10])

plot_matrix(np.real(kernel_matrix_fft), 'kernel_matrix_fft.eps')

In [ ]:
image_vect_fft = np.fft.fft(image_vect.flatten())
print(image_vect_fft)

plt.matshow(np.real(kernel_matrix_fft))
plt.colorbar()

plt.matshow(np.imag(kernel_matrix_fft))
plt.colorbar()

fft_result_fourier_vect = kernel_matrix_fft.dot(image_vect_fft)

matlab_result = 1.0e+02 * np.array(
    [[-1.8600 + 0.0000j],
     [ 0.0666 + 0.4632j],
     [-0.8000 + 0.0693j],
     [-0.6500 + 1.4500j],
     [ 0.4800 + 0.5543j],
     [-0.1066 + 0.1168j],
     [ 0.5200 + 0.0000j],
     [-0.1066 - 0.1168j],
     [ 0.4800 - 0.5543j],
     [-0.6500 - 1.4500j],
     [-0.8000 - 0.0693j],
     [ 0.0666 - 0.4632j]]).reshape(fft_result_fourier_vect.shape)

print('non equal elements: ')
diff = matlab_result - fft_result_fourier_vect
print(diff[np.abs(diff) > 1e-10])
    
plot_matrix(np.real(fft_result_fourier_vect).reshape((-1, 1)), 'fft_result_fourier_vect.eps', 'gray') 
    
result_fourier_vect = np.real(np.fft.ifft(fft_result_fourier_vect))

plot_matrix(np.real(result_fourier_vect).reshape((-1, 1)), 'result_fourier_vect.eps', 'gray', vmin=vmin, vmax=vmax) 

result_fourier_vect = result_fourier_vect[::-1]
result_fourier = result_fourier_vect.reshape(S)

plot_matrix(result_fourier, 'result_fourier.eps', 'gray', vmin=vmin, vmax=vmax) 

print(result_orig)
print(result_fourier)
assert np.allclose(result_fourier[1:, :result_orig.shape[1]], result_orig)

## Practical implementation

In [ ]:
from psf2otf import psf2otf

otf_kernel = psf2otf(kernel, S)
fft_image = np.fft.fft2(image)
plot_matrix(np.real(fft_image), 'fft_image.eps', 'gray')
fft_result_psf2otf = np.multiply(otf_kernel, fft_image)
plot_matrix(np.real(fft_result_psf2otf), 'fft_result_psf2otf.eps', 'gray')

result_psf2otf = np.fft.ifft2(fft_result_psf2otf)

plot_matrix(np.real(result_psf2otf), 'real_result_psf2otf.eps', 'gray', vmin=vmin, vmax=vmax)

assert np.allclose(result_psf2otf[:result_orig.shape[0], :result_orig.shape[1]], result_orig)

In [ ]:
# Visualization of PSF2OTF

from psf2otf import zero_pad

psf = np.array([[-1, -2], [-3, -4]])
shape = S

inshape = psf.shape

# Pad the PSF to outsize
psf = zero_pad(psf, shape, position='corner')
plot_matrix(psf, 'psf2otf_1.eps', vmax=2.0)

# Circularly shift OTF so that the 'center' of the PSF is
# [0,0] element of the array
for axis, axis_size in enumerate(inshape):
    psf = np.roll(psf, -int(axis_size / 2), axis=axis)

plot_matrix(psf, 'psf2otf_2.eps', vmax=2.0)

# Compute the OTF
otf = np.fft.fft2(psf)
plot_matrix(np.real(otf))
plot_matrix(np.imag(otf))
plot_matrix(np.abs(otf), 'psf2otf_3.eps')

# Estimate the rough number of operations involved in the FFT
# and discard the PSF imaginary part if within roundoff error
# roundoff error  = machine epsilon = sys.float_info.epsilon
# or np.finfo().eps
n_ops = np.sum(psf.size * np.log2(psf.shape))
otf = np.real_if_close(otf, tol=n_ops)

In [ ]:
array = np.array([
    [0.8147, 0.6324, 0.9575, 0.9572],
    [0.9058, 0.0975, 0.9649, 0.4854],
    [0.1270, 0.2785, 0.1576, 0.8003],
    [0.9134, 0.5469, 0.9706, 0.1419]])

matlab_fft = np.array([
  [ 9.7515 + 0.0000j,  -0.2897 + 0.8294j,   1.8715 + 0.0000j,  -0.2897 - 0.8294j],
  [ 1.9984 + 0.1191j,   0.6807 - 0.1951j,   0.9769 - 0.0926j,  -0.9050 + 0.1989j],
  [-0.3012 + 0.0000j,  -0.0571 + 0.8637j,  -3.0944 + 0.0000j,  -0.0571 - 0.8637j],
  [ 1.9984 - 0.1191j,  -0.9050 - 0.1989j,   0.9769 + 0.0926j,   0.6807 + 0.1951j]])

numpy_fft = np.fft.fft2(array)

diff = matlab_fft - numpy_fft
print(diff[np.abs(diff) > 1e-10])